<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import os
import sys
sys.path.insert(0, os.path.relpath('../')) 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from super_simulation.super_simulation_factory import SuperSimulationFactory
from transition_table import TransitionTable
from spark_policy import SparkPolicy
from utils.spark_bq_utils import upload_spark_model_inputs
import pandas as pd
from functools import partial

pd.set_option('display.max_rows', 500)

In [3]:
# load population data

total_pop = pd.read_csv("../state/CA/CA_total_pop_by_crime_category.csv")
total_pop = total_pop.iloc[:, :6]

# load admissions data

admissions = pd.read_csv("../state/CA/CA_admissions_by_crime_category.csv")
admissions = admissions.iloc[:, :6]

,Date,Crimes Against Persons,Property Crimes,Drug Crimes,Other Crimes,Missing
0,January 2018,"98,544","13,755","5,621","11,637",NaN
1,February 2018,"98,444","13,798","5,615","11,546",NaN
2,March 2018,"98,599","13,716","5,628","11,706",NaN
3,April 2018,"98,313","13,617","5,604","11,464",NaN
4,May 2018,"98,465","13,590","5,587","11,677",NaN


In [30]:
# Pre-processing plan:
# 1. dates to date format
# 2. numbers to int, get rid of commas
# 3. 2020-2021 "missing" column + missing data

In [4]:
# create month and year columns for total_pop

month_year = total_pop.Date.str.split(" " , n=1, expand = True)
total_pop["Month"], total_pop["Year"] = month_year.iloc[:, 0], month_year.iloc[:, 1]
total_pop = total_pop.drop("Date", axis = 1)

total_pop.Month = total_pop.index % 12 + 1

# another way to do this:
# month_to_number = {"January": 1, "February": 2, ...}
# total_pop.Month = total_pop.Month.map(month_to_number)

# a third way to do this:
# for i in range (len(total_pop)): 
#    total_pop.iloc[i,5] = i % 12 + 1

# create month and year columns for admissions table

month_year_admissions = admissions.Date.str.split(" " , n=1, expand = True)
admissions["Month"], admissions["Year"] = month_year_admissions.iloc[:, 0], month_year_admissions.iloc[:, 1]
admissions = admissions.drop("Date", axis = 1)

admissions.Month = admissions.index % 12 + 1

In [44]:
# remove commas from columns

def remove_commas(df, column_name):
    df[f"{column_name}"] = df[f"{column_name}"].apply(lambda x: x.replace(",", ""))

remove_commas(total_pop, "Crimes Against Persons")
remove_commas(total_pop, "Property Crimes")
remove_commas(total_pop, "Drug Crimes")
remove_commas(total_pop, "Other Crimes")

remove_commas(admissions, "Crimes Against Persons")
remove_commas(admissions, "Property Crimes")
remove_commas(admissions, "Drug Crimes")
remove_commas(admissions, "Other Crimes")

In [63]:
# change types of total_pop columns

total_pop["Crimes Against Persons"] = total_pop["Crimes Against Persons"].astype("int64")
total_pop["Property Crimes"] = total_pop["Property Crimes"].astype("int64")
total_pop["Drug Crimes"] = total_pop["Drug Crimes"].astype("int64")
total_pop["Other Crimes"] = total_pop["Other Crimes"].astype("int64")
total_pop["Year"] = total_pop["Year"].astype("int64")

# what should we do with NaN values in total_pop["Missing"]?

In [67]:
admissions.dtypes

Crimes Against Persons    object
Property Crimes           object
Drug Crimes               object
Other Crimes              object
Missing                   object
Month                      int64
Year                      object
dtype: object

In [72]:
# in admissions table: x means no number was reported, * means that the number was <10 so CA dashboard didn't give actual number
# options: 
# we could drop the rows with x's in them
# we could replace the x's with 0?
# we could use the perceding and subsequent year's data to replace each x

# for replacing *'s, we could replace them with any number [0,10]
